In [2]:
!pip install sentence_transformers
import pandas as pd
import numpy as np

from sentence_transformers import SentenceTransformer
from scipy.spatial.distance import cosine

  Using cached sentence_transformers-2.5.1-py3-none-any.whl.metadata (11 kB)
  Using cached transformers-4.38.2-py3-none-any.whl.metadata (130 kB)
  Using cached torch-2.2.1-cp310-cp310-manylinux1_x86_64.whl.metadata (26 kB)
  Using cached huggingface_hub-0.21.3-py3-none-any.whl.metadata (13 kB)
  Using cached fsspec-2024.2.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached typing_extensions-4.10.0-py3-none-any.whl.metadata (3.0 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 k

In [3]:
model = SentenceTransformer('all-MiniLM-L6-v2')

In [4]:
import boto3

# Initialize Boto3 client
s3 = boto3.client('s3')

# Define S3 bucket and file key
bucket_name = 'seeyoul8rallidater'
file_key = 'okcupid_profiles.csv'

# Specify local file path to save the dataset
local_file_path = 'okcupid.csv'

# Download the dataset from S3 to local SageMaker Studio instance
s3.download_file(bucket_name, file_key, local_file_path)
df = pd.read_csv("okcupid.csv")
print("Dataset downloaded successfully!")


Dataset downloaded successfully!


In [5]:
df_sample = df.sample(n=10000)
df_sample

,age,status,sex,orientation,body_type,diet,drinks,drugs,education,ethnicity,...,essay0,essay1,essay2,essay3,essay4,essay5,essay6,essay7,essay8,essay9
55435,25,single,f,bisexual,fit,strictly vegetarian,socially,sometimes,graduated from college/university,white,...,"my name is charlotte, but i go by charlie. i t...","making things, keeping fit and trying to figur...","making coffee, talking/storytelling, cooking, ...","hair - it's constantly changing. eyes - yes, t...",don't get me started...,1. pens 2. notebooks 3. coffee 4. my tongue 5....,"money, coffee, books, food, making things, bur...",typical? what's that? my current schedule mean...,"i'm from new jersey. shhh, no one else knows.",you know where we'd go on our first date.
31346,26,single,f,straight,curvy,mostly vegetarian,socially,sometimes,graduated from college/university,white,...,"friendly, genuine, energetic, generous, creati...","just finished art school, gainfully employed.",building/making stuff. cooking you the best di...,"eyes, they are really green.",books: tom robbins is the best. another roadsi...,"art, tactile experiences, cast iron pans, pho,...",diy. vintage cameras. thrift stores. and robots,NaN,NaN,you made it this far.
23575,27,single,m,straight,fit,strictly anything,socially,sometimes,graduated from masters program,white,...,i'm always weary of these things...i don't thi...,recently moved to sf from la (with a brief sto...,sleeping...in the morning...after the alarm ha...,"""hey, are you that guy from flight of the conc...","books: anything by vonnegut, murakami, bukowsk...",NaN,why i think so much,"out and about, getting into trouble",i'm hairy and i wear it proudly,"you're awesome (if i don't reply, i'm sorry -..."
37715,54,single,m,straight,used up,strictly vegetarian,not at all,sometimes,dropped out of college/university,white,...,"since you won't look at the questions section,...",writing something here that no one will read. ...,"nothing, really. i'm really bad at a few thing...",is usually nothing. i'm not noticed. if i fail...,no education therefore haven't the ability to ...,i'm already doing without most of them.,"you, now that you've visited my profile.",at rock band rehearsal.,if that's not obvious by now...... (i guess y...,you're having a bad time and society's enforce...
58384,35,single,m,straight,fit,anything,socially,never,graduated from college/university,asian,...,"i'm intelligent, passionate, funny, active and...",that's the big question for all of us. 2012 w...,caring about other people and solving problems...,for those that meet me for the first time...th...,"i love to read, before i moved to my new place...","friends and family...even on the worse days, i...",where life takes me. i feel like there is a go...,"either having a dinner and drinks, out with fr...",i not know...not too many skeletons in the clo...,"you're passionate, funny, fun-loving and easy ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43905,43,single,m,straight,average,mostly anything,socially,never,graduated from college/university,white,...,NaN,NaN,"communicating, listening, living spontaneously...",funny easy going guy!,i just love the tv series californication and ...,"family and friends, social events, travelling,...",what's the next phase in life... how to be pre...,out with some friends for a nice dinner and a ...,NaN,you are up for some new adventures.... and lik...
33153,30,single,m,straight,rather not say,NaN,rarely,never,graduated from high school,white,...,well ..... lol im a laid back good hearted guy...,im workin on that but my dream is to be a fabr...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7678,56,single,m,straight,average,mostly anything,rarely,never,dropped out of college/university,white,...,"i was raised in an ""ozzie and harriet"" family....",i make more money than anyone should make. i h...,jazz guitar. can i say that i'm good at it? no...,my hair and my eyes. this must be a girl thing...,"orwell's anima

In [6]:
# All essays but essay9, which is "you should message me if..."

df_demographics = df_sample[df_sample.columns.drop(list(df_sample.filter(regex="essay")))]
essays_df = df_sample.loc[:, ["essay0", "essay1", "essay2", "essay3", "essay4", 
                   "essay5", "essay6", "essay7", "essay8"]]
essays_df = essays_df.fillna(" ").astype(str)

essays_df.loc[:, "all_essays"] = essays_df.apply(" ".join, axis = 1)

df_all = pd.concat([df_demographics, essays_df.loc[:, ["all_essays"]]], axis = 1)

In [7]:
for e in df_all.all_essays[:10]:
    print(e)
    print("NEW BIO")

my name is charlotte, but i go by charlie. i tend to be loud and outgoing, but definitely appreciate my downtime. i freelance with my art skillz that i paid too much for. i make notebooks(both custom and my own line) and other assorted things such at mini top hats and black light jellyfish. i need to read more, and so do you.  i am usually hungry, sarcastic, and stoned. making things, keeping fit and trying to figure it all out.  currently moonlighting as a vegetarian butcher. making coffee, talking/storytelling, cooking, procrastinating, cutting out the bullshit in life, dancing and sing loudly in public without caring, listening, making things happen. hair - it's constantly changing. eyes - yes, they're blue, i grew them. how loud and talkative i am. don't get me started... 1. pens 2. notebooks 3. coffee 4. my tongue 5. my hands 6. my feet money, coffee, books, food, making things, burning man. typical? what's that? my current schedule means i don't have a monday through friday... i 

In [8]:
# Extract all essays from the DataFrame into a list
essays_list = df_all["all_essays"].tolist()

# Encode all essays using the model
embedding_list = model.encode(essays_list)

embedding_list = embedding_list.tolist()
# Create a Pandas Series with embeddings and indices
embedding_series = pd.Series(embedding_list, index=df_all.index, name="embedding")


In [9]:
embedding_series

55435    [0.020514823496341705, -0.09198534488677979, 0...
31346    [-0.03974509984254837, -0.09139120578765869, -...
23575    [0.018866078928112984, -0.0564219132065773, 0....
37715    [0.11587301641702652, -0.05597995966672897, 0....
58384    [0.002988251158967614, -0.07680206745862961, 0...
                               ...                        
43905    [0.08724702894687653, -0.07293721288442612, 0....
33153    [0.000331356655806303, -0.07900834828615189, 0...
7678     [-0.02853960171341896, -0.014243010431528091, ...
2534     [0.07088251411914825, -0.0943324938416481, 0.0...
58823    [0.0614720843732357, -0.07788537442684174, 0.0...
Name: embedding, Length: 10000, dtype: object

In [10]:
embedding_series.to_csv('embedding_series.csv') 

export_bucket_name = 'seeyoul8rallidater'
export_file_name = 'embedding_series.csv'
s3.upload_file('embedding_series.csv', export_bucket_name, export_file_name)

embedding_series

55435    [0.020514823496341705, -0.09198534488677979, 0...
31346    [-0.03974509984254837, -0.09139120578765869, -...
23575    [0.018866078928112984, -0.0564219132065773, 0....
37715    [0.11587301641702652, -0.05597995966672897, 0....
58384    [0.002988251158967614, -0.07680206745862961, 0...
                               ...                        
43905    [0.08724702894687653, -0.07293721288442612, 0....
33153    [0.000331356655806303, -0.07900834828615189, 0...
7678     [-0.02853960171341896, -0.014243010431528091, ...
2534     [0.07088251411914825, -0.0943324938416481, 0.0...
58823    [0.0614720843732357, -0.07788537442684174, 0.0...
Name: embedding, Length: 10000, dtype: object

In [11]:
print(df_all.loc[:,'sex'].unique())
print(df_all.loc[:,'orientation'].unique())


['f' 'm']
['bisexual' 'straight' 'gay']


In [12]:
df_all['pref_gen'] = np.where((df_all[ 'sex'] == 'm') & (df_all['orientation'] == 'gay'), 'm', 
                        np.where((df_all['sex'] == 'm') & (df_all['orientation'] == 'straight'), 'f',
                        np.where((df_all['sex'] == 'f') & (df_all['orientation'] == 'straight'), 'm',
                        np.where((df_all['sex'] == 'f') & (df_all['orientation'] == 'gay'), 'f',
                        np.where((df_all['orientation'] == 'bisexual'), 'all', 'no one')))))
print(df_all.loc[:,'pref_gen'].unique())

['all' 'm' 'f']


In [13]:
def compute_cosine_similarity(target_vector, vectors):
    similarities = []
    for vector in vectors:
        similarity = 1 - cosine(target_vector, vector)  # 1 - cosine distance to get cosine similarity
        similarities.append(similarity)
    return similarities

In [14]:
def rank_matches(input_row, pref_age_lower=False, pref_age_higher=False, min_similarity_score = 0.65):
    df_possible = df_all.copy()
    if pref_age_higher:
        df_possible = df_possible[df_possible['age'] <= df_possible.loc[input_row, 'age'] + pref_age_higher]
    if pref_age_lower:
        df_possible = df_possible[df_possible["age"] >= df_possible.loc[input_row,'age'] - pref_age_lower]
    if df_possible.loc[input_row,'pref_gen'] != 'all':
        df_possible = df_possible.loc[df_possible.loc[input_row,'pref_gen'] == df_possible.loc[:,'sex'],:]

    user_embeddings = embedding_series[input_row]

    other_embeddings = [embedding_series[i] for i in df_possible.index]
    # Compute the cosine similarity between the user's weighted embedding vector and all possible matches
    cosine_similarities = compute_cosine_similarity(user_embeddings, other_embeddings)
    # Recover index to match back to original dataframe
    similarity_scores = [(df_possible.index[index], score) for index, score in enumerate(cosine_similarities) if score >= min_similarity_score and score != 1]
    # Sort by similarity
    ranked_similarity = sorted(similarity_scores, key = lambda x: x[1], reverse = True)

    return ranked_similarity

In [74]:
test = rank_matches(55435, 10, 10)

In [75]:
test 

[(44841, 0.7564983819663028),
 (58778, 0.7529181019116709),
 (28135, 0.7487697498306617),
 (1476, 0.7420482823819853),
 (2737, 0.7387233385239826),
 (49513, 0.7345840520079907),
 (13654, 0.7335393554146233),
 (40685, 0.7310856298869725),
 (48794, 0.7308877544589562),
 (37003, 0.7271674465653315),
 (2630, 0.7263498937590707),
 (46171, 0.7245627629940274),
 (58740, 0.7239456107364587),
 (59380, 0.7211093591899137),
 (19, 0.7179516464496454),
 (17554, 0.7173452176206332),
 (36991, 0.7143907872257355),
 (32409, 0.7133460479340298),
 (16352, 0.713295129761346),
 (29353, 0.7100101189851652),
 (34537, 0.708432320869611),
 (16213, 0.7066558548290018),
 (30003, 0.706200131272224),
 (42574, 0.7055264361587844),
 (13272, 0.7045661059131029),
 (31659, 0.7041378950410188),
 (1862, 0.7038301359734581),
 (18614, 0.7027699064000057),
 (28517, 0.7020181657264594),
 (19576, 0.701691223285819),
 (26829, 0.701668270948996),
 (995, 0.7009221266880206),
 (26066, 0.7002942168054606),
 (5606, 0.70020031845770

In [15]:
df_all['matches'] = [rank_matches(i,10,10) for i, row in df_all.iterrows()]


In [18]:
df_all['matches']

55435    [(57837, 0.7229513275538625), (32304, 0.720553...
31346                                                   []
23575    [(3166, 0.7403998292957432), (53856, 0.7286884...
37715                                                   []
58384    [(45182, 0.7625276593086683), (27134, 0.716687...
                               ...                        
43905    [(23614, 0.7525679914843514), (18595, 0.744302...
33153                                                   []
7678                                                    []
2534     [(58947, 0.6952529094780014), (28146, 0.693300...
58823    [(55830, 0.7022291961748466), (33991, 0.698595...
Name: matches, Length: 10000, dtype: object

In [17]:
df_all['matches'].to_csv('okcupid_matches.csv') 

export_bucket_name = 'seeyoul8rallidater'
export_file_name = 'okcupid_matches.csv'

# Upload the exported CSV file to S3
s3.upload_file('okcupid_matches.csv', export_bucket_name, export_file_name)

print("DataFrame exported to S3 successfully!")

DataFrame exported to S3 successfully!


In [19]:
def rank_new_input(input_str, pref_gender=False, pref_age_lower=False, pref_age_higher=False, min_similarity_score = 0.5):
    df_possible = df_all.copy()
    if pref_gender:
        df_possible = df_possible.loc[df_possible.loc[:,'sex'] == pref_gender, :]
    if pref_age_higher:
        df_possible = df_possible[df_possible.loc[:, "age"] <= pref_age_higher]
    if pref_age_lower:
        df_possible = df_possible[df_possible.loc[:, "age"] >= pref_age_lower]
    user_embeddings = model.encode(input_str)
    other_embeddings = [embedding_series[i] for i in df_possible.index]
    
    # Compute the cosine similarity between the user's weighted embedding vector and all possible matches
    cosine_similarities = compute_cosine_similarity(user_embeddings, other_embeddings)
    # Recover index to match back to original dataframe
    similarity_scores = [(df_possible.index[index], score) for index, score in enumerate(cosine_similarities) if score >= min_similarity_score and score != 1]
    # Sort by similarity
    ranked_similarity = sorted(similarity_scores, key = lambda x: x[1], reverse = True)
    return ranked_similarity

In [20]:
rank_new_input("I like bacon and pumpkin pie",'f', 30,40, 0.2)


[(41044, 0.4228414440522055),
 (8442, 0.40899515086068106),
 (56353, 0.4002669883044079),
 (57393, 0.3992225632961809),
 (57807, 0.36974330397736976),
 (4909, 0.3632609735818413),
 (10869, 0.36213544891805927),
 (25341, 0.35448643169689276),
 (2140, 0.35128419816244505),
 (17548, 0.3399951689064553),
 (361, 0.33657003944391106),
 (45420, 0.3265008189596521),
 (26142, 0.3253343873590181),
 (56485, 0.3250035666564459),
 (47211, 0.32015171962220534),
 (4869, 0.3135948065605232),
 (4985, 0.3116498551656287),
 (37363, 0.30528361910241264),
 (999, 0.30497832575185235),
 (2969, 0.3044974265760828),
 (8780, 0.30429316749318325),
 (46823, 0.30214197961017997),
 (31183, 0.2970314947698399),
 (32550, 0.2954427131962165),
 (41392, 0.29399662161167983),
 (15843, 0.2910477550688292),
 (49900, 0.2906661977832674),
 (8557, 0.2902958139338465),
 (9372, 0.28963554782873546),
 (58844, 0.28942050802625485),
 (38938, 0.2880726858189203),
 (44106, 0.28720397382768115),
 (54930, 0.28621548104805083),
 (48578